<a href="https://colab.research.google.com/github/harnalashok/deeplearning/blob/main/Reusing_trained_layers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Last amended: 26th January, 2021
# My folder: https://github.com/harnalashok/deeplearning
# Objective: 
#             Reuse layers from one neural network in another.
#             Layers were trained using different objects
# Ref:
#      Aurelien Book: Page 347 
#      Save and Laod models: https://www.tensorflow.org/guide/keras/save_and_serialize

**Problem**<br>
We will take fashion-mnist data. This data contains ten items. We will filter out data for two of the items--shirts and sandals. Prepare a predictive model with remaining eight items.<br>
After predictive model is made, we will use the trained layers of this model in another NN model to make predictions for the remaining two items. That is, we will reuse trained layers from the first model in the IInd model.

In [ ]:
# 1.0 Call libraries
import tensorflow as tf
from tensorflow import keras
from keras.models import Model
from tensorflow.keras.datasets import fashion_mnist
import numpy as np

In [ ]:
# 1.1 Display multiple commands outputs from a cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
# 2.0 Get split data
(X_train, y_train), (X_test,y_test) = fashion_mnist.load_data()

4423680/4422102 [==============================] - 0s 0us/step


In [ ]:
# 2.1 Train data shape
X_train.shape    # (60000, 28, 28)
y_train.shape    # (60000,)

(60000, 28, 28)

(60000,)

In [ ]:
# 2.2 Fashion items
items = ["T-shirt/top", "Trouser", "Pullover", "Dress", "Coat", "Sandal", "Shirt", "Sneaker", "Bag", "Ankle boot"]

In [ ]:
# 2.3 Normalise data
X_train = X_train/255.0
X_test = X_test/255.0

In [ ]:
# 2.4 We separate our data further
#     Shirts and Sandals in one dataset and remaining 8 items
#     in another dataset. Shirts and Sandals are at index 5 & 6
#     Data with shirts and sandals:
# 2.4.1
ss_train = X_train[np.logical_or (y_train == 5,y_train == 6), :, :]
ss_train.shape   # (12000, 28, 28)
# 2.4.2
ss_y_train = y_train[np.logical_or (y_train == 5,y_train == 6)]
ss_y_train.shape  # (12000,)

# 2.4.3
ss_test = X_test[np.logical_or (y_test == 5,y_test == 6), :, :]
ss_test.shape    # (2000, 28, 28)
# 2.4.4
ss_y_test = y_test[np.logical_or (y_test == 5,y_test == 6)]
ss_y_test.shape   # (2000,)

(12000, 28, 28)

(12000,)

(2000, 28, 28)

(2000,)

In [ ]:
# 3.0 Change labels of shirts and sandals
#     in ss_y_train/ss_y_test to 1 and 0

ss_y_train[ss_y_train == 6] = 1
ss_y_train[ss_y_train == 5] = 0

# 3.1 Recheck 
ss_y_train

In [ ]:
# 4.0 train/test data for remaining 8 items
# 4.1 Train data
data_train = X_train[ np.logical_and (y_train != 5 , y_train !=6), :,:]
data_train.shape  # (48000, 28, 28)
# 4.2 Train labels
data_y_train = y_train[ np.logical_and (y_train != 5 , y_train !=6)]
data_y_train.shape  # (48000,)

# 4.3 Test data
data_test = X_test[ np.logical_and (y_test != 5 , y_test !=6), :,:]
data_test.shape   # (8000, 28, 28)
# 4.4 Test labels
data_y_test = y_test[ np.logical_and (y_test != 5 , y_test !=6)]
data_y_test.shape  # (8000,)

(48000, 28, 28)

(48000,)

(8000, 28, 28)

(8000,)

In [ ]:
# 4.5 Reshape train/test data
#     8-items data
data_train = data_train.reshape(-1,784)
data_test = data_test.reshape(-1,784)
# 4.6 2-items data
ss_train = ss_train.reshape(-1,784)
ss_test = ss_test.reshape(-1,784)

In [ ]:
# 5.0 Our Ist model. No frills

units =100

# 5.1 Input layer
input = keras.layers.Input(shape = (784,))

# 5.2 Hidden layers
x = keras.layers.Dense(units = units, activation = "selu", name = "Layer_a")(input)
x = keras.layers.Dense(units = units, activation = "selu", name = "Layer_b")(x)
x = keras.layers.Dense(units = units, activation = "selu", name = "Layer_c")(x)

# 5.3 Output layer
out = keras.layers.Dense(units = 10, activation = 'softmax', name = "Layer_d")(x)

# 5.4 Design model
model = Model(inputs = [input], outputs = [out])

# 5.5 Compile model
model.compile(loss = "sparse_categorical_crossentropy", metrics = "accuracy")

# 5.6 Fit the model
model.fit(data_train,data_y_train, epochs = 20)

Epoch 1/20
1500/1500 [==============================] - 4s 2ms/step - loss: 0.4795 - accuracy: 0.8307
Epoch 2/20
1500/1500 [==============================] - 4s 2ms/step - loss: 0.2641 - accuracy: 0.9058
Epoch 3/20
1500/1500 [==============================] - 4s 2ms/step - loss: 0.2303 - accuracy: 0.9184
Epoch 4/20
1500/1500 [==============================] - 4s 2ms/step - loss: 0.2160 - accuracy: 0.9232
Epoch 5/20
1500/1500 [==============================] - 4s 2ms/step - loss: 0.2011 - accuracy: 0.9278
Epoch 6/20
1500/1500 [==============================] - 4s 2ms/step - loss: 0.1905 - accuracy: 0.9318
Epoch 7/20
1500/1500 [==============================] - 4s 3ms/step - loss: 0.1836 - accuracy: 0.9340
Epoch 8/20
1500/1500 [==============================] - 4s 3ms/step - loss: 0.1763 - accuracy: 0.9374
Epoch 9/20
1500/1500 [==============================] - 4s 2ms/step - loss: 0.1706 - accuracy: 0.9399
Epoch 10/20
1500/1500 [==============================] - 4s 3ms/step - loss: 0.164

In [ ]:
# 5.7 Summary
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 784)]             0         
_________________________________________________________________
a (Dense)                    (None, 100)               78500     
_________________________________________________________________
b (Dense)                    (None, 100)               10100     
_________________________________________________________________
c (Dense)                    (None, 100)               10100     
_________________________________________________________________
d (Dense)                    (None, 10)                1010      
Total params: 99,710
Trainable params: 99,710
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# 6.0 Save this model in a folder. 
#     Folder will be created if 
#     it does not exist:

tf.saved_model.save(model, "/root/model")

INFO:tensorflow:Assets written to: /root/model/assets


In [ ]:
# 6.1 Now that model is saved, 
#     we can delete it from RAM
del(model)

In [ ]:
# 6.2 Check if deleted
model.layers
model.layers[3]

NameError: ignored

In [ ]:
for layer in model.layers[:4]:
  layer.trainable = False

In [ ]:
# Let us load thesaved model now
model_l = keras.models.load_model("/root/model")

In [ ]:
# Check summary
model_l.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 784)]             0         
_________________________________________________________________
Layer_a (Dense)              (None, 100)               78500     
_________________________________________________________________
Layer_b (Dense)              (None, 100)               10100     
_________________________________________________________________
Layer_c (Dense)              (None, 100)               10100     
_________________________________________________________________
Layer_d (Dense)              (None, 10)                1010      
Total params: 99,710
Trainable params: 99,710
Non-trainable params: 0
_________________________________________________________________


In [ ]:
xy = keras.models.Model(model_l.layers[:-1])
out = keras.layers.Dense(units = 2, activation = 'softmax', name = "d")


AttributeError: ignored

In [ ]:
for layer in model_l.layers[:4]:
  layer.trainable = False

In [ ]:
units =100
input = keras.layers.Input(shape = (784,))
x = keras.layers.Dense(units = units, activation = "selu", name = "a")(input)
x = keras.layers.Dense(units = units, activation = "selu", name = "b")(x)
x = keras.layers.Dense(units = units, activation = "selu", name = "c")(x)
out = keras.layers.Dense(units = 2, activation = 'softmax', name = "d")(x)

In [ ]:
model_1 = Model(inputs = [input], outputs = [out])


In [ ]:
model.layers[1].get_weights()

[array([[ 0.0200765 ,  0.03928816,  0.02216594, ...,  0.08818798,
          0.0273401 , -0.01568698],
        [ 0.06713179,  0.03235041, -0.04764441, ..., -0.01043751,
          0.02157476,  0.10488354],
        [-0.33216465,  0.251126  , -0.18246882, ...,  0.3922871 ,
          0.297408  ,  0.25099432],
        ...,
        [ 0.0466023 , -0.1216006 , -0.01429964, ...,  0.26163495,
         -0.16638726,  0.5435451 ],
        [ 0.30099398, -0.329225  ,  0.16592345, ...,  0.07812608,
          0.11534598,  0.227801  ],
        [ 0.00073423, -0.07167973, -0.2865736 , ...,  0.09619594,
          0.24174218,  0.23470792]], dtype=float32),
 array([-2.36573309e-01, -2.35280357e-02,  1.04582310e-02, -2.32576556e-03,
        -8.40267241e-02,  2.33466566e-01, -1.06402397e-01, -1.96787328e-01,
         3.09968114e-01, -4.44545858e-02, -4.51167047e-01,  3.58818546e-02,
        -1.44800574e-01, -4.93491516e-02,  9.34602171e-02,  5.79757988e-02,
         5.08762226e-02, -2.65835792e-01, -3.08039069e

In [ ]:
for r in range(4):
  model_1.layers[r].set_weights(model.layers[r].get_weights())

In [ ]:
model_1.compile(loss = "sparse_categorical_crossentropy", metrics = "accuracy")

In [ ]:
model_1.fit(ss_train,ss_y_train,epochs = 10)


Epoch 1/10
375/375 [==============================] - 1s 2ms/step - loss: 0.0316 - accuracy: 0.9933
Epoch 2/10
375/375 [==============================] - 1s 3ms/step - loss: 0.0032 - accuracy: 0.9995
Epoch 3/10
375/375 [==============================] - 1s 2ms/step - loss: 0.0031 - accuracy: 0.9996
Epoch 4/10
375/375 [==============================] - 1s 2ms/step - loss: 0.0013 - accuracy: 0.9998
Epoch 5/10
375/375 [==============================] - 1s 2ms/step - loss: 8.0996e-04 - accuracy: 0.9998
Epoch 6/10
375/375 [==============================] - 1s 2ms/step - loss: 2.0326e-04 - accuracy: 0.9999
Epoch 7/10
375/375 [==============================] - 1s 3ms/step - loss: 1.9579e-05 - accuracy: 1.0000
Epoch 8/10
375/375 [==============================] - 1s 2ms/step - loss: 7.9210e-04 - accuracy: 0.9999
Epoch 9/10
375/375 [==============================] - 1s 2ms/step - loss: 4.5399e-09 - accuracy: 1.0000
Epoch 10/10
375/375 [==============================] - 1s 2ms/step - loss: 1.986